     Pierre-François SAUNIER
     Clara ALLOIS


# PROJET NUMERIQUE: LIGNES DE NIVEAU


# 1ère méthode : par dichotomie (PF modifie)

## Contour simple:




### Amorce : 



f étant continûment différentiable, elle est continue, ainsi on est certain qu'il existe un t dans [0,1]  tq f(0,t)=c si (f(0,1)-c)*(f(0,0)-c)<0 d'après le théorème des valeurs intermédiaires.



In [ ]:
def find_seed(f,c=0,eps=2**-26):
    
    def g(x,y):
        return f(x,y)-c
    a=0
    b=1
    if g(0,0)*g(0,1)>0:
        return None
    
    while b-a>=eps:
        mil=(a+b)/2
        if g(0,a)*g(0,mil)<=0:
            b=mil
        else : 
            a=mil
    return a



In [ ]:
def f(x,y): return x**2+y**2
find_seed(f,0.5,2**-26)

0,707**2=0.5 : ok ça fonctionne (ici x=0)

### Propagation:

####  on balaye régulièrement la figure avec un pas régulier suffisament petit et on utilise la dichotomie.

In [ ]:

def find_seed2(f,abscisse,c=0,eps=2**-26):
    
    def g(x,y):
        return f(x,y)-c
    
    a=0
    b=1
    if g(abscisse,0)*g(abscisse,1)>0:
        return None
    
    while b-a>=eps:
        mil=(a+b)/2
        if g(abscisse,a)*g(abscisse,mil)<=0:
            b=mil
        else : 
            a=mil
    return a



def simple_contour(f,c=0.0,delta=0.01,eps=2**-26):
    pas=delta/sqrt(2)
    abscisse=0.0
    X=[]
    Y=[]
    while abscisse<=1 and find_seed2(f,abscisse,c,eps)!=None:
        X.append(abscisse)
        Y.append(find_seed2(f,abscisse,c,eps))
        abscisse+=pas
        
    return (X,Y)




In [ ]:
import matplotlib.pyplot as plt

##1er exemple
def f(x,y):
    return x**2+y**2

(X,Y)=simple_contour(f,c=1.0,delta=0.01,eps=2**-26)
plt.plot(X,Y)
plt.show()



In [ ]:
import matplotlib.pyplot as plt

##2eme exemple
def f(x,y):
    return (exp(-x**2-y**2)-exp(-(x-1)**2-(y-1)**2))

(X,Y)=simple_contour(f,c=0.5,delta=0.001,eps=2**-26)
plt.plot(X,Y)


(X1,Y1)=simple_contour(f,c=0,delta=0.001,eps=2**-26)
plt.plot(X1,Y1)


(X2,Y2)=simple_contour(f,c=0.25,delta=0.001,eps=2**-26)
plt.plot(X2,Y2)

plt.show()





Commentaires: Cette méthode fonctionne mais présente des inconvénients :

-couteuse car non optimisée

-la méthode de dichotomie permet de détecter un seul zéro d'où pb si la ligne fait des boucles.

Il faut donc redéfinir une méthode dichotomique ou utliser une autre méthode.
On a choisit d'utiliser une méthode de Newton pour cibler plus précisément la recherche.

## Contour complexe

#### Dichotomie brute, on balaye horizontalement et verticalement chaque carré qui forment la portion rectangulaire

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

def find_seed_ordonnee(f,abscisse,ybas,yhaut,c,eps=2**(-26)): #dichotomie verticale à une abscisse fixée avec une limite basse et une limite haute
    
    def g(x,y):
        return (f(x,y)-c)
    
    if g(abscisse,ybas)*g(abscisse,yhaut)>0:
        return(None)
    a=ybas
    b=yhaut
    
    while b-a >= eps:
        mil=(a+b)/2
        if g(abscisse,a)*g(abscisse,mil)<=0:
            b=mil
        else:
            a=mil
    return(b)


##############
def find_seed_abscisse(f,ordonnee,xgauche,xdroit,c,eps=2**(-26)): #dichotomie horizontale à une ordonée fixée avec une limite basse et une limite haute
    
    
    def g(x,y):
        return (f(x,y)-c)
    
    if g(xgauche,ordonnee)*g(xdroit,ordonnee)>0:
        return(None)
    a=xgauche
    b=xdroit
    
    while b-a >= eps:
        mil=(a+b)/2
        if g(a,ordonnee)*g(mil,ordonnee)<=0:
            b=mil
        else:
            a=mil
    return(b)

    
#######
def contour(f,xgauche,xdroit,ybas,yhaut,c,delta=0.01,eps=2**-13):
    
    pas = delta / np.sqrt(2)
    
    
    
    abscisse = xgauche
    X1=[]
    Y1=[]
    while abscisse<=xdroit and find_seed_ordonnee(f,abscisse,ybas,yhaut,c,eps)!=None:
        X1.append(abscisse)
        Y1.append(find_seed_ordonnee(f,abscisse,ybas,yhaut,c,eps))
        abscisse=abscisse + pas
    plt.plot(X1,Y1)
    # on balaye verticalement de xgauche à xdroite 
    
    
    
    
    abscisse = xdroit
    X2=[]
    Y2=[]
    while abscisse>=xgauche and find_seed_ordonnee(f,abscisse,ybas,yhaut,c,eps)!=None:
        X2.append(abscisse)
        Y2.append(find_seed_ordonnee(f,abscisse,ybas,yhaut,c,eps))
        
        abscisse=abscisse - pas
    plt.plot(X2,Y2)
    # on balaye verticalement de xdroit à xgauche 
    
    
    
    ordonnee = ybas
    X3=[]
    Y3=[]
    while ordonnee<=yhaut and find_seed_abscisse(f,ordonnee,xgauche,xdroit,c,eps)!=None:
        X3.append(find_seed_abscisse(f,ordonnee,xgauche,xdroit,c,eps))
        Y3.append(ordonnee)
        ordonnee=ordonnee + pas
    plt.plot(X3,Y3)
    #on balaye horizontalement de ybas à yhaut
    

    ordonnee = yhaut
    X4=[]
    Y4=[]
    while ordonnee>=ybas and find_seed_abscisse(f,ordonnee,xgauche,xdroit,c,eps)!=None:
        X4.append(find_seed_abscisse(f,ordonnee,xgauche,xdroit,c,eps))
        Y4.append(ordonnee)
        ordonnee=ordonnee - pas
    plt.plot(X4,Y4)
    #on balaye horizontalement de yhaut à ybas

    


In [ ]:

def f(x,y):
    return (exp(-x**2-y**2)-exp(-(x-1)**2-(y-1)**2))

####
cote = [-1 , 0 , 1 , 2 ]
for i in range(len(cote)-1):
    for j in range(len(cote)-1):
        for c in [-1.5 , -1 , -0.5 , 0 , 0.5 , 1 , 1.5 ] :
            contour(f,cote[i],cote[i+1],cote[j],cote[j+1],c)
#on cherche pour la fonction f les lignes de niveau entre -1,5 et 1,5.
#pour ce faire on découpe la figure en plusieurs carrés dont les abscisses et ordonnes limites sont listées dans cote


Rq: ce programme ne donne pas toutes les lignes de niveau sur mon ordi(Clara) mais sur celui de PF on obtient une figure plus complète...

# 2ème méthode moins naïve : en utilisant Newton

In [ ]:
import autograd
import autograd.numpy as np
from math import *
from matplotlib import pyplot as plt
###


def gradient(f,x,y): 
    g = autograd.grad
    return np.r_[g(f,0)(x,y) , g(f,1)(x,y)]




On reprend la méthode du poly (en annexe) pour calculer le gradient

In [ ]:

def newton(f,x0,n=10):
    x = x0
    derivee=autograd.grad(f)
    for i in range(n):
        x=x-f(x)/derivee(x)
    return x



Recherche de zéro par la méthode de Newton

In [ ]:

def point_suivant(f,c,x0,y0,delta):
    grad = gradient(f,x0,y0)                                #on calcule le gradient en (x0,y0)
    grad = grad / sqrt( grad[0]**2 + grad[1]**2)           #on normalise le gradient
    tangente = (grad[1] , -grad[0] )                        # la tangente est normale au gradient. Si le gradient vaut (a,b) la tangente est (-b,a)
    theta0 = atan(tangente[1]/tangente[0])                 #on récupère l'angle correspondant à théta0
    
    
    
    def g(theta):                                       #g paramétrise f sur le cercle de rayon delta et de centre (x0,y0)
        return f(x0+delta * np.cos(theta) , y0 + delta * np.sin(theta) ) - c
    thetafinal = newton(g , theta0)
    
    return (x0 + delta * np.cos(thetafinal) , y0 + delta * np.sin(thetafinal) ) #renvoie le point suivant de notre ligne de niveau







on calcule le point suivant de notre ligne de niveau 

In [ ]:

x0,y0 = -0.5 , 0.0
X=[x0]
Y=[y0]
x,y = x0 , y0
for i in range(40):
    x,y = point_suivant(f,f(x0,y0),x,y,0.1)
    X.append(x)
    Y.append(y)
plt.clf()
plt.plot(X,Y)
plt.show()

